# 理解简化版SMO算法

## Platt的SMO算法

**算法目标：**求出一系列alpha和b，一旦求出了这些alpha，很容易计算出权重向量w，并得到分隔超平面。  
**工作原理：**每次循环中选择两个alpha进行优化处理。一旦找到一对合适的alpha，那么就增大其中一个同时减小另一个。合适的条件是这两个alpha必须要在间隔边界之外，还没有进行过区间化处理或者不在边界上。

In [1]:
import random
import numpy as np

### SMO算法中的辅助函数

In [2]:
# 创建数据集和特征向量
def loadDataSet(fileName):
    dataMat = []; labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat, labelMat

In [3]:
# i表示alpha的下标，m表示所有alpha的总数
# 随机取出一个不为i的下标
def selectJrand(i, m):
    j = i
    while (j == i):
        j = int(random.uniform(0, m))
    return j

In [4]:
# 用于调整大于H或者小于L的alpha值
def clipAlpha(aj, H, L):
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj

In [5]:
dataArr, labelArr = loadDataSet('testSet.txt')

In [6]:
labelArr

[-1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0]

可以看到，这个数据集中采用的类别标签是-1和1，而不是0和1

### 简化版SMO算法

**SMO函数的伪代码**  
创建一个alpha向量并将其初始化为0的向量  
当迭代次数小于最大迭代次数时（外循环）  
&emsp;&emsp;对数据集中的每个数据向量（内循环）:  
&emsp;&emsp;&emsp;&emsp;如果该数据向量可以被优化:  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;随机选择另外一个数据向量  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;同时优化这两个向量  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;如果两个向量都不能被优化，退出内循环  
&emsp;&emsp;如果所有向量都没被优化，增加迭代数目，继续下一次循环

In [7]:
# dataMatIn-数据集， classLabels-类别标签，C-常数， toler-容错率，maxIter-退出前最大的循环次数
def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    # 进行Numpy矩阵转化
    dataMatrix = np.mat(dataMatIn); labelMat = np.mat(classLabels).transpose()
    # 得到m行n列
    b = 0; m,n = np.shape(dataMatrix)
    # 构建一个alpha列矩阵，初始化为0
    alphas = np.mat(np.zeros((m,1)))
    # 在没有任何alpha改变的情况下遍历数据集的次数
    iter = 0
    # 当iter达到输入值，函数结束运行并退出
    while (iter < maxIter):
        # 每次循环前，将alphaPairsChanged设置为0
        alphaPairsChanged = 0
        for i in range(m):
            # 预测出来的分类 yi=sum(alpha_i*yi(xi*xj))+b
            fXi = float(np.multiply(alphas,labelMat).T * (dataMatrix * dataMatrix[i,:].T)) + b
            # 预测与真实的分类误差
            Ei = fXi - float(labelMat[i])
            if ((labelMat[i] * Ei < -toler) and (alphas[i] < C)) or ((labelMat[i] * Ei > toler) and (alphas[i] > 0)):
                # 误差很大，可以对该数据实例对应的alpha值进行优化
                # 随机选择第二个不同于i的值j
                j = selectJrand(i,m)
                # 计算j对应的预测分类 
                fXj = float(np.multiply(alphas,labelMat).T * (dataMatrix * dataMatrix[j,:].T)) + b
                # 计算j的预测与真实分类的误差
                Ej = fXj - float(labelMat[j])
                # 调用浅拷贝重新分配内存空间
                alphaIold = alphas[i].copy(); alphaJold = alphas[j].copy();
                # 计算L和H
                if (labelMat[i] != labelMat[j]):
                    # 如果不属于同一分类
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    # 同属于一个分类
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L==H: print("L==H"); continue
                # eta是alpha[j]的最优修改量
                eta = 2.0 * dataMatrix[i,:] * dataMatrix[j,:].T - dataMatrix[i,:] * dataMatrix[i,:].T - dataMatrix[j,:] * dataMatrix[j,:].T
                # 如果eta为正的，则退出当前for循环
                if eta >= 0: print("eta>=0"); continue
                # 调整alphas[j]
                alphas[j] -= labelMat[j] * (Ei - Ej) / eta
                alphas[j] = clipAlpha(alphas[j],H,L)
                # 不需要优化的条件
                if (abs(alphas[j] - alphaJold) < 0.00001): print("j not moving enough"); continue
                # 调整alphas[i]，修改量与j相同，但方法相反
                alphas[i] += labelMat[j] * labelMat[i] * (alphaJold - alphas[j])
                # 设置常数项b
                b1 = b - Ei- labelMat[i] * (alphas[i] - alphaIold) * dataMatrix[i,:] * dataMatrix[i,:].T - labelMat[j] * (alphas[j] - alphaJold) * dataMatrix[i,:] * dataMatrix[j,:].T
                b2 = b - Ej- labelMat[i] * (alphas[i] - alphaIold) * dataMatrix[i,:] * dataMatrix[j,:].T - labelMat[j] * (alphas[j] - alphaJold) * dataMatrix[j,:] * dataMatrix[j,:].T
                if (0 < alphas[i]) and (C > alphas[i]): b = b1
                elif (0 < alphas[j]) and (C > alphas[j]): b = b2
                else: b = (b1 + b2) / 2.0
                alphaPairsChanged += 1
                print("iter: %d i:%d, pairs changed %d" % (iter, i, alphaPairsChanged))
        if (alphaPairsChanged == 0): iter += 1
        else: iter = 0
        print("iteration number: %d" % iter)
    return b,alphas

In [8]:
b, alphas = smoSimple(dataArr, labelArr, 0.6, 0.001, 40)

iter: 0 i:0, pairs changed 1
iter: 0 i:2, pairs changed 2
L==H
L==H
L==H
L==H
iter: 0 i:10, pairs changed 3
L==H
L==H
L==H
j not moving enough
iter: 0 i:38, pairs changed 4
L==H
j not moving enough
iter: 0 i:54, pairs changed 5
iter: 0 i:65, pairs changed 6
j not moving enough
j not moving enough
j not moving enough
L==H
L==H
j not moving enough
iteration number: 0
j not moving enough
L==H
j not moving enough
L==H
iter: 0 i:10, pairs changed 1
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
iter: 0 i:90, pairs changed 2
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
L==H
j not moving enough
L==H
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving

iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 3
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 4
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
iteration number: 5
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 6
j not moving enough
L==H
j not moving enough
j not moving enough
iter: 6 i:69, pairs changed 1
iteration number: 0
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving en

iteration number: 7
j not moving enough
j not moving enough
j not moving enough
iteration number: 8
j not moving enough
j not moving enough
j not moving enough
iteration number: 9
j not moving enough
j not moving enough
j not moving enough
iteration number: 10
j not moving enough
j not moving enough
j not moving enough
iteration number: 11
j not moving enough
j not moving enough
j not moving enough
iteration number: 12
j not moving enough
j not moving enough
j not moving enough
iteration number: 13
j not moving enough
j not moving enough
j not moving enough
iteration number: 14
j not moving enough
iter: 14 i:29, pairs changed 1
j not moving enough
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
iter: 2 i:54, pairs changed 1
j not moving e

j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 5
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 6
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 7
iter: 7 i:23, pairs changed 1
j not moving enough
iter: 7 i:54, pairs changed 2
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
iter: 2 i:55, pairs changed 1
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number: 3
j not moving enough
j not movi

j not moving enough
j not moving enough
j not moving enough
iteration number: 4
iter: 4 i:29, pairs changed 1
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
iteration number: 3
j not moving enough
j not moving enough
j not moving enough
iteration number: 4
j not moving enough
iter: 4 i:52, pairs changed 1
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not moving enough
iteration number: 3
j not moving enough
j not moving enough
j not moving enough
iteration number: 4
j not moving enough
j not moving enough
j not moving enough
iteration number: 5
j not moving enough


j not moving enough
iteration number: 4
j not moving enough
j not moving enough
j not moving enough
iteration number: 5
j not moving enough
j not moving enough
j not moving enough
iteration number: 6
j not moving enough
j not moving enough
j not moving enough
iteration number: 7
j not moving enough
j not moving enough
j not moving enough
iteration number: 8
j not moving enough
j not moving enough
j not moving enough
iteration number: 9
j not moving enough
j not moving enough
j not moving enough
iteration number: 10
j not moving enough
j not moving enough
j not moving enough
iteration number: 11
j not moving enough
j not moving enough
j not moving enough
iteration number: 12
j not moving enough
j not moving enough
iter: 12 i:52, pairs changed 1
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
j not 

j not moving enough
iteration number: 3
j not moving enough
j not moving enough
iteration number: 4
j not moving enough
j not moving enough
iteration number: 5
j not moving enough
j not moving enough
iteration number: 6
j not moving enough
j not moving enough
iteration number: 7
j not moving enough
j not moving enough
iteration number: 8
iter: 8 i:29, pairs changed 1
j not moving enough
j not moving enough
iteration number: 0
j not moving enough
j not moving enough
iteration number: 1
j not moving enough
j not moving enough
iteration number: 2
j not moving enough
j not moving enough
iteration number: 3
j not moving enough
j not moving enough
iteration number: 4
j not moving enough
j not moving enough
iteration number: 5
j not moving enough
j not moving enough
iteration number: 6
j not moving enough
j not moving enough
iteration number: 7
j not moving enough
j not moving enough
iteration number: 8
j not moving enough
j not moving enough
iteration number: 9
j not moving enough
j not movi

In [9]:
b

matrix([[-3.86210797]])

In [10]:
alphas[alphas>0]

matrix([[0.12620243, 0.24553751, 0.00291327, 0.36882666]])

In [11]:
# 支持向量的个数
np.shape(alphas[alphas>0])

(1, 4)

In [12]:
# 支持向量的数据点
for i in range(100):
    if alphas[i] > 0.0:
        print(dataArr[i] , labelArr[i])

[4.658191, 3.507396] -1.0
[3.457096, -0.082216] -1.0
[5.286862, -2.358286] 1.0
[6.080573, 0.418886] 1.0
